https://leetcode.com/problems/elimination-game/

In [ ]:
import torch as T
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from tqdm import trange, tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from IPython import display
import time

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set()

In [ ]:
def lastRemaining(n: int) -> int:
    xs = list(range(1, n+1))

    while len(xs) > 1:
        xs = xs[1::2][::-1]

    return xs[0]

n = 10_000
x, y = map(lambda a: np.log2(np.array(a)), zip(*[(x, lastRemaining(x)) for x in trange(1, n+1)]))

In [ ]:
x = T.from_numpy(x)
y = T.from_numpy(y)

In [ ]:
class Net(nn.Module):
    def __init__(self, h):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, h, bias=True),
            # nn.LeakyReLU(negative_slope=0.01, inplace=True),
            
            *[nn.Linear(h, h, bias=True)] * 2,
            
            
            nn.Linear(h, 1, bias=True),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.net(x)
    

dl = DataLoader(TensorDataset(x, y), batch_size=1024, shuffle=False)

net = Net(h=64)

opt = optim.Adam(net.parameters(), lr=1e-3)
# opt = optim.SGD(net.parameters(), lr=1e-4)

crit = nn.MSELoss()
# crit = nn.L1Loss()

sched = optim.lr_scheduler.MultiStepLR(opt, milestones=[10, 50], gamma=0.1, verbose=True)
# sched = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=100//4, eta_min=1e-5, verbose=True)
# sched = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', patience=5, verbose=True)

avgs = []

for e in range(1, 100 + 1):   
    avg = 0
    net.train()
    
    for i, (X, Y) in enumerate(dl):    
        opt.zero_grad()
        
        y_ = net(X.unsqueeze(1).float()).squeeze(1)
        
        loss = crit(y_, Y.float())
        # loss = T.mean((y_**2 - Y.float()**2) ** 2)
        
        loss.backward()
        opt.step()
        
        avg += loss.item() / len(dl)

    sched.step()
    avgs.append(avg)
    
    net.eval()
    display.clear_output(wait=True)
    # plt.clf()
    plt.figure(figsize=(16, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, y)
    with T.no_grad():
        plt.plot(x, T.round(net(x.unsqueeze(1).float()).squeeze(1)))
    plt.title(f"{e}")
    plt.subplot(1, 2, 2)
    plt.plot(avgs, linestyle='--', marker='.', markersize=10)
    plt.show()